# Scenario 1: A single data scientist participating in an ML competition

Scenerio setup: 
+ Tracking server: no.
+ Backend store: local filesystem.
+ Artifact store: local filesystem.
The experiment can be explored locally by launching the ML flow UI.

In [37]:
import mlflow

In [38]:
print(f"tracking URI: {mlflow.get_tracking_uri()}")

tracking URI: sqlite:///mlflow.db


In [42]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:///D:/code/nda-mlops-zoomcamp-2024/week-02/mlruns/1', creation_time=1717343602995, experiment_id='1', last_update_time=1717343602995, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='file:///D:/code/nda-mlops-zoomcamp-2024/week-02/mlruns/0', creation_time=1717343602987, experiment_id='0', last_update_time=1717343602987, lifecycle_stage='active', name='Default', tags={}>]

## Create an experiment and log a new run

In [44]:
mlflow.end_run()

In [45]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():
    X, y = load_iris(return_X_y=True)
    
    params = { "C": 0.1, "random_state": 42 }
    mlflow.log_params(params)
    
    lr = LogisticRegression(**params)
    lr = lr.fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))
    
    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifact URI: {mlflow.get_artifact_uri()}") 

default artifact URI: file:///D:/code/nda-mlops-zoomcamp-2024/week-02/mlruns/2/05e1247c0ded4579835ec988a587c5a2/artifacts


In [47]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:///D:/code/nda-mlops-zoomcamp-2024/week-02/mlruns/2', creation_time=1717843484176, experiment_id='2', last_update_time=1717843484176, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='file:///D:/code/nda-mlops-zoomcamp-2024/week-02/mlruns/1', creation_time=1717343602995, experiment_id='1', last_update_time=1717343602995, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='file:///D:/code/nda-mlops-zoomcamp-2024/week-02/mlruns/0', creation_time=1717343602987, experiment_id='0', last_update_time=1717343602987, lifecycle_stage='active', name='Default', tags={}>]

## Interacting with the Model Registry

In [48]:
from mlflow.client import MlflowClient

client = MlflowClient()

In [51]:
from mlflow.exceptions import MlflowException

try:
    print(client.search_registered_models())
except MlflowException:
    print("It's not possible to access the model registry")

[<RegisteredModel: aliases={'in-preview': 1, 'staging': 1}, creation_timestamp=1717688401070, description='', last_updated_timestamp=1717688401106, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1717688401106, current_stage='None', description='', last_updated_timestamp=1717688401106, name='nyc-taxi-model', run_id='92025337761b47258bb509b166789a29', run_link='', source='file:///D:/code/nda-mlops-zoomcamp-2024/week-02/mlruns/1/92025337761b47258bb509b166789a29/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>], name='nyc-taxi-model', tags={}>]


It would not be possible to access model registry if you are using backend store as the local filesystem.<br>
To do so, we should have the backend store as the others such as sqlite,...